# モデルの最大コンテキスト長を超えるテキストの埋め込み

OpenAIの埋め込みモデルは、最大長を超えるテキストを埋め込むことができません。最大長はモデルによって異なり、文字列の長さではなく_トークン_で測定されます。トークン化に馴染みがない場合は、[tiktokenでトークンを数える方法](How_to_count_tokens_with_tiktoken.ipynb)をご確認ください。

このノートブックでは、モデルの最大コンテキスト長を超えるテキストを処理する方法を説明します。`text-embedding-3-small`からの埋め込みを使用してデモンストレーションを行いますが、同じアイデアは他のモデルやタスクにも適用できます。埋め込みについて詳しく学ぶには、[OpenAI Embeddings Guide](https://beta.openai.com/docs/guides/embeddings)をご確認ください。

## 1. モデルのコンテキスト長

まず、モデルを選択し、APIから埋め込みを取得する関数を定義します。

In [2]:
from openai import OpenAI
import os
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt, retry_if_not_exception_type

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

EMBEDDING_MODEL = 'text-embedding-3-small'
EMBEDDING_CTX_LENGTH = 8191
EMBEDDING_ENCODING = 'cl100k_base'

# let's make sure to not retry on an invalid request, because that is what we want to demonstrate
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6), retry=retry_if_not_exception_type(openai.BadRequestError))
def get_embedding(text_or_tokens, model=EMBEDDING_MODEL):
    return client.embeddings.create(input=text_or_tokens, model=model).data[0].embedding

`text-embedding-3-small`モデルは`cl100k_base`エンコーディングで8191トークンのコンテキスト長を持っており、その制限を超えるとエラーが発生することがわかります。

In [4]:
long_text = 'AGI ' * 5000
try:
    get_embedding(long_text)
except openai.BadRequestError as e:
    print(e)

Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 10001 tokens (10001 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}


明らかに、特に大量の埋め込みをプログラムで処理する際には、これらのエラーを避けたいと考えています。しかし、最大コンテキスト長よりも長いテキストに直面する可能性があります。以下では、これらの長いテキストを処理するための主要なアプローチについて説明し、レシピを提供します：（1）テキストを許可される最大長まで単純に切り詰める方法、および（2）テキストをチャンクに分割し、各チャンクを個別に埋め込む方法。

## 1. 入力テキストの切り詰め

最もシンプルな解決策は、入力テキストを許可される最大長まで切り詰めることです。コンテキスト長はトークン単位で測定されるため、切り詰める前にまずテキストをトークン化する必要があります。APIはテキスト形式とトークン形式の両方の入力を受け付けるため、適切なエンコーディングを使用していることに注意すれば、トークンを文字列形式に戻す必要はありません。以下は、そのような切り詰め関数の例です。

In [3]:
import tiktoken

def truncate_text_tokens(text, encoding_name=EMBEDDING_ENCODING, max_tokens=EMBEDDING_CTX_LENGTH):
    """Truncate a string to have `max_tokens` according to the given encoding."""
    encoding = tiktoken.get_encoding(encoding_name)
    return encoding.encode(text)[:max_tokens]

先ほどの例が、今度はエラーなしで動作します。

In [4]:
truncated = truncate_text_tokens(long_text)
len(get_embedding(truncated))

1536

## 2. 入力テキストのチャンク化

切り捨てが機能するとはいえ、関連する可能性のあるテキストを破棄することは明らかな欠点です。別のアプローチとして、入力テキストをチャンクに分割し、各チャンクを個別に埋め込む方法があります。その後、チャンクの埋め込みを個別に使用するか、各チャンクのサイズで重み付けした平均化などの方法で組み合わせることができます。

シーケンスをチャンクに分割する関数を[Pythonの公式クックブック](https://docs.python.org/3/library/itertools.html#itertools-recipes)から使用します。

In [5]:
from itertools import islice

def batched(iterable, n):
    """Batch data into tuples of length n. The last batch may be shorter."""
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := tuple(islice(it, n))):
        yield batch

次に、文字列をトークンにエンコードし、それをチャンクに分割する関数を定義します。

In [6]:
def chunked_tokens(text, encoding_name, chunk_length):
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    chunks_iterator = batched(tokens, chunk_length)
    yield from chunks_iterator

最後に、入力テキストが最大コンテキスト長よりも長い場合でも、入力トークンをチャンクに分割し、各チャンクを個別に埋め込むことで、埋め込みリクエストを安全に処理する関数を書くことができます。`average`フラグを`True`に設定すると、チャンク埋め込みの重み付き平均を返し、`False`に設定すると、変更されていないチャンク埋め込みのリストをそのまま返します。

In [7]:
import numpy as np


def len_safe_get_embedding(text, model=EMBEDDING_MODEL, max_tokens=EMBEDDING_CTX_LENGTH, encoding_name=EMBEDDING_ENCODING, average=True):
    chunk_embeddings = []
    chunk_lens = []
    for chunk in chunked_tokens(text, encoding_name=encoding_name, chunk_length=max_tokens):
        chunk_embeddings.append(get_embedding(chunk, model=model))
        chunk_lens.append(len(chunk))

    if average:
        chunk_embeddings = np.average(chunk_embeddings, axis=0, weights=chunk_lens)
        chunk_embeddings = chunk_embeddings / np.linalg.norm(chunk_embeddings)  # normalizes length to 1
        chunk_embeddings = chunk_embeddings.tolist()
    return chunk_embeddings

再び、長い入力テキストを処理できるようになりました。

In [8]:
average_embedding_vector = len_safe_get_embedding(long_text, average=True)
chunks_embedding_vectors = len_safe_get_embedding(long_text, average=False)

print(f"Setting average=True gives us a single {len(average_embedding_vector)}-dimensional embedding vector for our long text.")
print(f"Setting average=False gives us {len(chunks_embedding_vectors)} embedding vectors, one for each of the chunks.")


Setting average=True gives us a single 1536-dimensional embedding vector for our long text.
Setting average=False gives us 2 embedding vectors, one for each of the chunks.


場合によっては、テキストの意味を保持するために、段落の境界や文の境界でチャンクを分割することが理にかなっている場合があります。